In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn import svm
from sklearn import neighbors
from sklearn import linear_model

In [2]:
np.random.seed(42)

In [3]:
historical_data = pd.read_csv('historical_data.csv')

In [6]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [36]:
frame_test = historical_data[['store_id','store_primary_category']].groupby(by=['store_id','store_primary_category'],dropna=False).value_counts()
frame_test

store_id  store_primary_category
1         mexican                    8
2         japanese                   5
3         fast                       1
          salad                      2
4         asian                      1
                                    ..
6986      fast                      35
          indian                     1
          other                      1
          pasta                      1
6987      american                   2
Length: 15435, dtype: int64

In [63]:
store_id_unique = historical_data["store_id"].unique().tolist()
store_id_and_category = {store_id: historical_data[historical_data.store_id == store_id].store_primary_category.mode() 
                         for store_id in store_id_unique}

In [64]:
def fill(store_id):
    """Return primary store category from the dictionary"""
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan

# fill null values
historical_data["clean_store_primary_category"] = historical_data.store_id.apply(fill)

In [50]:
## note that the number of non-null store_id values is less than the number of non-null 
# store_primary category values. We can probably fill in most of the missing values via inference

## there are also stores with multiple store_primary_category values. Such stores might be 'ghost kitchens',
# and need to be backfilled appropriately
# TODO #1 write function to downfill NaN store_primary_category values

def fillNulls_store_category(historical_data:pd.DataFrame) -> pd.DataFrame:
    store_id_list = historical_data['store_id'].unique().tolist()
    store_id_cat = {store_id: historical_data[historical_data.store_id == store_id].store_primary_category.mode() for store_id in store_id_list}
    historical_data['clean_store_primary_category']
    try:
        for x in store_id_list:
            historical_data['clean_store_primary_category'] = historical_data.store_id.apply(store_id_cat.values[0])
    except Exception as ee:
        raise ee
    return historical_data


In [11]:
## update types
## create target variable
def addFeatures_durations(historical_data:pd.DataFrame) -> pd.DataFrame:
    try:
        historical_data['created_at'] = pd.to_datetime(historical_data['created_at'])
        historical_data['actual_delivery_time'] = pd.to_datetime(historical_data['actual_delivery_time'])
        ## calculate delivery duration
        historical_data['actual_total_delivery_duration'] = (historical_data['actual_delivery_time'] - historical_data['created_at']).dt.total_seconds()
        ## estimated time spent outside the store/not on order preparation
        historical_data['est_time_non-prep'] = historical_data['estimated_order_place_duration'] + historical_data['estimated_store_to_consumer_driving_duration']
        ## estimated time spent in the store/not driving
        historical_data['est_time_prep'] = historical_data['actual_total_delivery_duration'] - historical_data['est_time_non-prep']
    except Exception as ex:
        raise ex
    return historical_data

In [67]:
## add ratio features
def addFeatures_ratios(historical_data:pd.DataFrame) -> pd.DataFrame:
    try:
        historical_data['busy_to_onshift'] = historical_data['total_busy_dashers'] / historical_data['total_onshift_dashers']
        # drop infinite values
        historical_data['busy_to_onshift'].replace([np.inf,-np.inf],np.nan,inplace=True)
    except Exception as ex:
        raise ex
    return historical_data

In [85]:
## add dummies for categories
def addFeatures_dummies(historical_data:pd.DataFrame,dummy_column:str) -> pd.DataFrame:
    try:
        dumm = pd.get_dummies(historical_data[dummy_column],prefix=str(dummy_column + '_'),dtype=float)
        # concat dummies
        historical_data = pd.concat([historical_data,dumm],axis=1)
        historical_data = historical_data.drop(columns=[dummy_column])
    except Exception as ex:
        raise ex
    return historical_data